In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
color_pal = plt.rcParams['axes.prop_cycle'].by_key()['color']

import warnings
warnings.filterwarnings("ignore")
plt.style.use('ggplot')

In [2]:
# !rm -r kuma_utils
!git clone https://github.com/analokmaus/kuma_utils.git

fatal: destination path 'kuma_utils' already exists and is not an empty directory.


In [3]:
import sys
sys.path.append("kuma_utils/")
from kuma_utils.preprocessing.imputer import LGBMImputer

In [4]:
train = pd.read_csv("Data/train_allx.csv")
y_train = pd.read_csv("Data/train_y.csv")
test = pd.read_csv("Data/val_allx.csv")
y_test = pd.read_csv("Data/Sample_Output.csv", names = ['ID', 'Target'], header=None)

In [5]:
# Dropping ID column
train.drop('ID', axis=1, inplace=True)
test.drop('ID', axis=1, inplace=True)

In [22]:
cat_features = ['D_36', 'D_44']

In [13]:
train['Target'] = y_train.astype('str')

# Encode categorical and numerical features
For categorical features, label encoding will be applied. For numerical features, rank-gauss transform will be applied.

In [14]:
from xfeat import SelectCategorical, SelectNumerical
from kuma_utils.preprocessing.xfeat import Pipeline, TargetEncoder
from kuma_utils.preprocessing import DistTransformer
from category_encoders import OrdinalEncoder

In [15]:
# num_enc = Pipeline([
#     SelectNumerical(),
#     DistTransformer('rankgauss')
# ])

cat_enc = Pipeline([
    SelectCategorical(exclude_cols=['Target']),
    OrdinalEncoder(handle_missing='return_nan')
], target_col='Target')

In [17]:
train_cat,= cat_enc.fit_transform(train).astype(int, errors='ignore'), #num_enc.fit_transform(train).astype(float)
categorical_index = list(range(train_cat.shape[1]))
# train_encoded = pd.concat([train_cat, train_num], axis=1) 
# train_encoded.head()

In [20]:
test_cat = cat_enc.transform(test).astype(int, errors='ignore')
test_cat.head()

,D_36,D_44
0,1,1.0
1,2,2.0
2,1,1.0
3,1,3.0
4,1,3.0


In [21]:
train[['D_36', 'D_44']] = train_cat
test[['D_36', 'D_44']] = test_cat

# LGBM Imputer

In [25]:
%%time
lgbm_imtr = LGBMImputer(cat_features=cat_features, n_iter=200, verbose=True)

train_lgbmimp = lgbm_imtr.fit_transform(train.drop("Target", axis=1))
test_lgbmimp = lgbm_imtr.transform(test)

  0%|          | 0/109 [00:00<?, ?it/s]

  0%|          | 0/109 [00:00<?, ?it/s]

CPU times: user 2h 36min 49s, sys: 3min 10s, total: 2h 40min
Wall time: 23min 44s


In [26]:
train_lgbmimp

,B_37,S_24,S_4,S_14,B_25,D_38,B_30,D_138,P_2,S_7,...,D_43,D_4,D_85,D_78,D_42,D_52,D_29,R_1,R_28,S_5
0,0.006462,0.008118,0.853027,0.556641,1.003906,-0.024589,0.021301,0.511230,1.004883,0.081543,...,0.066916,0.502441,0.002375,0.008270,0.008835,0.001913,1.007812,0.008171,0.008514,0.004307
1,0.007748,0.006310,0.758301,0.686035,0.001836,-0.010097,0.088501,0.557129,0.687988,0.353027,...,0.099481,0.009544,1.002930,0.002613,0.003374,0.000530,0.006649,0.006798,0.003426,0.009224
2,0.005405,0.005219,0.766602,0.284668,1.002930,0.007511,0.005821,0.854980,0.927734,0.059479,...,-0.024789,0.008736,0.008751,0.002541,0.009300,0.001654,0.000212,0.007599,0.009224,0.009636
3,0.004135,0.000032,0.363281,0.004623,0.533691,0.041853,0.002443,0.526367,0.700684,0.334473,...,0.140374,0.004646,0.007095,0.000235,0.004581,0.004738,0.002653,0.001221,0.009529,0.007523
4,0.002968,0.002892,0.757324,0.427734,1.001953,-0.011410,0.008247,0.820312,1.001953,0.144287,...,0.086688,0.000344,0.005577,0.001367,0.005760,1.008789,1.003906,0.003401,0.001020,0.000535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458908,0.003967,0.005070,0.167480,0.421143,0.697266,-0.050492,0.059113,0.526855,0.852051,0.724121,...,-0.064142,0.005363,0.001710,0.001604,0.001387,0.009468,0.000110,0.009346,0.009995,0.001259
458909,0.006248,0.007683,0.003338,0.006954,1.004883,0.055898,0.001700,0.575195,0.885254,-0.000085,...,0.008311,0.008728,0.008682,0.008438,0.008392,0.006836,0.004223,0.008179,0.004337,0.003376
458910,0.004993,0.003204,0.167114,0.289307,0.650879,0.013336,0.007294,0.406738,0.632324,0.346924,...,-0.097511,0.002733,0.000283,0.009102,0.000087,0.007229,0.000245,0.007935,0.007233,0.007114
458911,0.002348,0.005432,0.330078,0.003138,0.689941,-0.026039,0.001930,0.417969,0.846191,0.172607,...,-0.039661,0.007492,0.009583,0.001172,0.005016,0.002506,0.009796,0.001634,0.009781,0.003271


In [32]:
trainNa = train.drop('Target', axis=1).isna()
trainNa.columns = [f"{c}_missing" for c in trainNa.columns]

In [35]:
train_imputed = pd.concat([train_lgbmimp, trainNa], axis=1)

In [36]:
testNa = test.isna()
testNa.columns = [f"{c}_missing" for c in testNa.columns]

In [37]:
test_imputed = pd.concat([test_lgbmimp, testNa], axis=1)

In [38]:
train_imputed.to_feather('Data/x_train__LGBMImputed.ftr')
test_imputed.to_feather('Data/x_test__LGBMImputed.ftr')